In [1]:
import warnings
import gc
import os
import random
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from datetime import datetime
from glob import glob
from tqdm import tqdm

warnings.filterwarnings(action = 'ignore')

In [2]:
train_2017 = []
train_2018 = []
train_2019 = []
train_2020 = []

for i, path in enumerate(sorted(glob("Data/train/KNOW*.csv"))):
    if i == 0:
        train_2017 = pd.read_csv(path)
        pass
    elif i==1:
        train_2018 = pd.read_csv(path)
        pass
    elif i==2:
        train_2019 = pd.read_csv(path)
        pass
    else:
        train_2020 = pd.read_csv(path)
        pass
    
test_2017 = []
test_2018 = []
test_2019 = []
test_2020 = []

for i, path in enumerate(sorted(glob("Data/test/KNOW*.csv"))):
    if i == 0:
        test_2017 = pd.read_csv(path)
        pass
    elif i==1:
        test_2018 = pd.read_csv(path)
        pass
    elif i==2:
        test_2019 = pd.read_csv(path)
        pass
    else:
        test_2020 = pd.read_csv(path)
        pass

### 2020년 데이터 결측치 처리

- 중앙값으로 결측치 처리
- data leakage 고려하여 train 데이터로 결측치 처리

In [3]:
for col in train_2020:
        train_2020[col].replace(' ', np.nan, inplace = True)
        if col != 'knowcode':
            test_2020[col].replace(' ', np.nan, inplace = True)

In [4]:
train_2020.loc[lambda x : (x['saq'+str(40)+'_2'].notnull()) & (x['saq'+str(40)+'_1'] == 1), 'saq40_2'] = np.nan
for i in range(1, 45):
    train_2020.loc[lambda x : (x['saq'+str(i)+'_2'].isnull()) & (x['saq'+str(i)+'_1'] == 1), 'saq'+str(i)+'_2'] = 0

test_2020.loc[lambda x : (x['saq'+str(40)+'_2'].isnull()) & (x['saq'+str(40)+'_1'] != 1), 'saq40_2'] = 4
for i in range(1, 45):
    test_2020.loc[lambda x : (x['saq'+str(i)+'_2'].isnull()) & (x['saq'+str(i)+'_1'] == 1), 'saq'+str(i)+'_2'] = 0

In [5]:
train_2020.loc[(train_2020['bq4'] == 2) & (train_2020['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
test_2020.loc[(test_2020['bq4'] == 2) & (test_2020['bq4_1a'].isnull()), 'bq4_1a'] = '없음'
train_2020.loc[(train_2020['bq4'] == 2) & (train_2020['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
train_2020.loc[(train_2020['bq4'] == 2) & (train_2020['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
test_2020.loc[(test_2020['bq4'] == 2) & (test_2020['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
test_2020.loc[(test_2020['bq4'] == 2) & (test_2020['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

In [6]:
train_2020.loc[(train_2020['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
train_2020.loc[(train_2020['bq4_1c'].isnull()), 'bq4_1c'] = '없음'
test_2020.loc[(test_2020['bq4_1b'].isnull()), 'bq4_1b'] = '없음'
test_2020.loc[(test_2020['bq4_1c'].isnull()), 'bq4_1c'] = '없음'

In [7]:
for col in test_2020:
    train_2020[col].replace('없다','없음', inplace = True)
    test_2020[col].replace('없다','없음', inplace = True)    

In [8]:
train_2020.loc[(train_2020['bq5'] == 2) & (train_2020['bq5_1'].isnull()),'bq5_1'] = 0
test_2020.loc[(test_2020['bq5'] == 2) & (test_2020['bq5_1'].isnull()),'bq5_1'] = 0

train_2020.loc[(train_2020['bq5'] == 2) & (train_2020['bq5_2'].isnull()),'bq5_2'] = '없음'
test_2020.loc[(test_2020['bq5'] == 2) & (test_2020['bq5_2'].isnull()),'bq5_2'] = '없음'

In [9]:
train_2020.loc[lambda x : x.bq26_1.isnull(), 'bq26_1'] = '없음'
test_2020.loc[lambda x : x.bq26_1.isnull(), 'bq26_1'] = '없음'

In [10]:
train_2020.loc[lambda x : (x.bq28.isnull()) & (x.bq27_1 == 1), 'bq28'] = 2
test_2020.loc[lambda x : (x.bq28.isnull()) & (x.bq27_1 == 1), 'bq28'] = 2
train_2020.loc[lambda x : (x.bq28.isnull()) & (x.bq27_1 == 2), 'bq28'] = 1
test_2020.loc[lambda x : (x.bq28.isnull()) & (x.bq27_1 == 2), 'bq28'] = 1

In [11]:
# total_2020 = pd.concat([train_2020, test_2020], axis=0)
total_2020 = train_2020
bq1_list = total_2020.bq1.unique()

for bq1 in bq1_list:
    mean_bq30_1 = round(total_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_1.notnull())].bq30_1.apply(lambda x: int(x)).mean())
    train_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_1.isnull()), 'bq30_1'] = mean_bq30_1
    test_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_1.isnull()), 'bq30_1'] = mean_bq30_1
    
    mean_bq30_2 = round(total_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_2.notnull())].bq30_2.apply(lambda x: int(x)).mean())
    train_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_2.isnull()), 'bq30_2'] = mean_bq30_2
    test_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_2.isnull()), 'bq30_2'] = mean_bq30_2
    
    try:
        mean_bq30_3 = round(total_2020.loc[lambda x: (x.bq1 == bq1) & (x.bq30_3.notnull())].bq30_3.apply(lambda x: int(x)).mean())
    except:
        mean_bq30_3 = mean_bq30_2
        
    train_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_3.isnull()), 'bq30_3'] = mean_bq30_3
    test_2020.loc[lambda x : (x.bq1 == bq1) & (x.bq30_3.isnull()), 'bq30_3'] = mean_bq30_3

In [12]:
train_2020['bq18_10'] = train_2020['bq18_10'].fillna('없음')
test_2020['bq18_10'] = test_2020['bq18_10'].fillna('없음')

### 2020년 데이터 object 컬럼 처리

In [13]:
convert = []
for col in train_2020.select_dtypes('object'):
    if train_2020[col].nunique() < 20:
        train_2020[col] = train_2020[col].astype(float)
        
for col in ['bq30_1', 'bq30_2', 'bq30_3']:
    train_2020[col] = train_2020[col].astype(float)

In [14]:
convert = []
for col in test_2020.select_dtypes('object'):
    if test_2020[col].nunique() < 20:
        test_2020[col] = test_2020[col].astype(float)
        
for col in ['bq30_1', 'bq30_2', 'bq30_3']:
    test_2020[col] = test_2020[col].astype(float)

In [15]:
train_2020.to_csv('Data/train/1.preprocessing_train_2020.csv', index = False)
test_2020.to_csv('Data/test/1.preprocessing_test_2020.csv', index = False)